# Modificacion de Tablas

Las tablas que tenemos son:
1. Activos: Informacion de cada activo
2. Historico Adjuntos Activos OT: Match de que activo corresponde a que .txt, e informacion de la orden de trabajo fuera del reporte
3. Registros ordenes de Trabajo: Informacion de ordenes de trabajo, incluidas los reportes en formato html

In [3]:
import numpy as np
import pandas as pd
#import prettytable as pt

In [5]:
activos = pd.read_csv('../Data/Activos.csv', delimiter=';')

In [7]:
hist_ots = pd.read_csv('../Data/HistoricoAdjuntosActivosOT.csv', delimiter=';')

In [13]:
ots = pd.read_excel('../Data/RegistrosOrdenesDeTrabajo.xlsx')

In [14]:
cols = pt.PrettyTable()
cols.add_column('Activos', activos.columns.tolist() + ['', '', ''])
cols.add_column('Hist OTs', hist_ots.columns.tolist()+ [''])
cols.add_column('OTs', ots.columns.tolist())
print(cols)

NameError: name 'pt' is not defined

# Agregamos a la tabla de Historico de ordenes de trabajo el Reporte en texto libre correspondiente

In [15]:
# Modificaciones al nombre de archivo para asegurar que sea como el del txt
hist_ots['DOCUMENT'] = hist_ots['DOCUMENT'].replace('ISTORIA', 'istoria', regex = True)
hist_ots['DOCUMENT'] = hist_ots['DOCUMENT'].apply(str.strip)

In [16]:
#Creamos una columna vacia por ahora
hist_ots['WORKLOG'] = ''

for i in range(len(hist_ots.DOCUMENT)):
    filename = hist_ots.DOCUMENT[i]         # Leemos el nombre de archivo para cada una
    
    try:
        lineas = []                             # Lista vacía para guardar el contenido
        with open (f'EPM_HISTORICOS_OT_ANDRES_OCAMPO\HISTORICOS_OT\{filename}.txt', 'rt') as worklog:  # Abre el .txt
            for linea in worklog:               # Lee linea por linea
                lineas.append(linea)            # Guarda la linea como elemento de la lista
        
        worklog = ''.join(lineas)               # Juntamos en un string

        hist_ots.WORKLOG[i] = worklog           # Asignamos el worklog a la entrada correspondiente

    except:
        i =+ 1 


In [17]:
# Reemplazamos los caracteres \n (new line) con espacios
hist_ots['WORKLOG'] = hist_ots['WORKLOG'].replace('\n', ' ', regex = True)

In [18]:
# Volvemos NA lo que quedo vacío
vacio_a_na = {'': np.nan}
hist_ots['WORKLOG'] = hist_ots['WORKLOG'].replace(vacio_a_na)

In [19]:
# Miramos la cantidad de vacios
hist_ots.WORKLOG.isna().sum()

58391

In [20]:
# Comparamos con los vacios del repositorio, con una muestra random de la mitad:
vacios = ['HistoriaJDE_OT_3815495.txt',
          'HistoriaJDE_OT_3816996.txt',
          'HistoriaJDE_OT_3818147.txt',
          'HistoriaJDE_OT_3818153.txt',
          'HistoriaJDE_OT_3821964.txt',
          'HistoriaJDE_OT_3824321.txt',
          'HistoriaJDE_OT_3824493.txt',
          'HistoriaJDE_OT_3825889.txt',
          'HistoriaJDE_OT_3826393.txt',
          'HistoriaJDE_OT_3829275.txt',
          'HistoriaJDE_OT_3832219.txt',
          'HistoriaJDE_OT_3834917.txt']

In [21]:
hist_ots.loc[hist_ots.DOCUMENT.isin(vacios)]

,ASSETNUM,SLXNUMACTEXT,SLXDESCACTEXT,WORKTYPE,SLXNUMTOT,SLXDESCOT,SLXFECCREACIONOT,SLXFECINICIOOT,SLXFECFINOT,DOCUMENT,WORKLOG


Segun el repositorio, hay 1301 vacios, que parece que no estaban en la tabla. Y parece que hay 9908 en la tabla que no coinciden con ningun .txt. 

(Total de archivos .txt - vacios) = (Total del filas del data frame - NaN en la columna de worklog)?

In [23]:
import os
import glob

#Obtenemos el total de los archivos
files = os.listdir('../Data/HISTORICOS_OT')

(len(files) - 1301) == (len(hist_ots.DOCUMENT) - 9908)

False

In [24]:
# Archivos con info en la carpeta
len(files) - 1301

53065

In [25]:
# Archivos con info en la tabla modificada
len(hist_ots.DOCUMENT) - 9908

48483

Segun esto hay unos 5k .txt que no estan en la tabla? 

In [26]:
#miramos los 9k que si estan pero no tienen .txt
hist_ots.DOCUMENT.loc[hist_ots.WORKLOG.isna()]

0        HistoriaJDE_01312033_OT_1000740
1        HistoriaJDE_01312033_OT_1001233
2        HistoriaJDE_01312033_OT_1001382
3        HistoriaJDE_01312033_OT_1001383
4        HistoriaJDE_01312033_OT_1001386
                      ...               
58386             HistoriaJDE_OT_3852163
58387             HistoriaJDE_OT_3852165
58388             HistoriaJDE_OT_3852167
58389             HistoriaJDE_OT_3852185
58390             HistoriaJDE_OT_3852186
Name: DOCUMENT, Length: 58391, dtype: object

In [27]:
faltantes = hist_ots.DOCUMENT.loc[hist_ots.WORKLOG.isna()]

good_docs = []
for doc in faltantes:
    if len(doc) < 27: good_docs.append(doc)

In [28]:
good_docs

['HistoriaJDE_OT_2213651',
 'HistoriaJDE_OT_2213652',
 'HistoriaJDE_OT_2213654',
 'HistoriaJDE_OT_2213663',
 'HistoriaJDE_OT_2214642',
 'HistoriaJDE_OT_2214646',
 'HistoriaJDE_OT_2214660',
 'HistoriaJDE_OT_2215272',
 'HistoriaJDE_OT_2215302',
 'HistoriaJDE_OT_2215307',
 'HistoriaJDE_OT_2215324',
 'HistoriaJDE_OT_2215338',
 'HistoriaJDE_OT_2215362',
 'HistoriaJDE_OT_2215388',
 'HistoriaJDE_OT_2215716',
 'HistoriaJDE_OT_2215728',
 'HistoriaJDE_OT_2215758',
 'HistoriaJDE_OT_2216251',
 'HistoriaJDE_OT_2216693',
 'HistoriaJDE_OT_2216926',
 'HistoriaJDE_OT_2217019',
 'HistoriaJDE_OT_2218520',
 'HistoriaJDE_OT_2219337',
 'HistoriaJDE_OT_2222652',
 'HistoriaJDE_OT_2222683',
 'HistoriaJDE_OT_2222968',
 'HistoriaJDE_OT_2227769',
 'HistoriaJDE_OT_2227833',
 'HistoriaJDE_OT_2227834',
 'HistoriaJDE_OT_2227836',
 'HistoriaJDE_OT_2227840',
 'HistoriaJDE_OT_2228223',
 'HistoriaJDE_OT_2228512',
 'HistoriaJDE_OT_2228874',
 'HistoriaJDE_OT_2229502',
 'HistoriaJDE_OT_2229541',
 'HistoriaJDE_OT_2229601',
 

In [29]:
# Parece que solo se está saltando ese por x o y motivo, lo agregamos y ya
hist_ots.WORKLOG.loc[hist_ots.DOCUMENT == 'HistoriaJDE_OT_3641553'] = 'Fecha: Noviembre 15/2017 Programo: Daniel Palacios O.Ejecuto: Jairo Torres y Frank Córdoba. Se realiza inspección y limpieza al barraje de salida, se realiza inspección y mantenimiento a mallas separadoras del barraje, baquelitas separadoras de barras, conexiones entre barras y estado del aislamiento, también se revisan el estado de los CTLs, de sus conexiones eléctricas en el tablero +CCCTS. Trabajo ejecutado por: Jairo Torres y Frank Córdoba. XE: LIMPIAR E INSPECCIONAR BARRAJE ESTATOR GENERADOR UNIDAD'


/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [30]:
hist_ots.WORKLOG.isna().sum()

58390

In [31]:
# Todos los nombres de nuestros .txt son de 26 caracteres, miramos que no haya quedado ninguno de esos sin worklog en la tabla
set(hist_ots.DOCUMENT.loc[hist_ots.WORKLOG.isna()].apply(len))

{22, 29, 30, 31}

In [32]:
# ahora si, guardo el csv
hist_ots.to_csv('historico_OTs_texto_libre.csv')


# Quitamos las tag html de la tabla de Historico reciente de OTs

In [35]:
import html2text

In [36]:
ots['WORKLOG'] = ots['WORKLOG'].apply(html2text.html2text)

In [37]:
# Reemplazamos los caracteres \n (new line) con espacios
ots['WORKLOG'] = ots['WORKLOG'].replace('\n', ' ', regex = True)

In [38]:
ots.head()

,WONUM,DESCRIPTION,ASSETNUM,WORKTYPE,REPORTDATE,ACTSTART,ACTFINISH,PROBLEM,CAUSE,REMEDY,WORKLOG
0,81187,R/V SOFTWARE SCADA LA TASAJERA,2977213,CPROG,2018-07-17 09:13:14.999,2018-08-01 10:58:12.000,2018-12-20 08:25:17.999,CONTROL DEFICIENTE,CABLE DE PUESTA A TIERRA SUELTO,CONECTAR,AJUSTE DE LA HORA EN SCADA. Lo primero que se...
1,81011,LAVAR JUEGO DE PLACAS,2976926,CPROG,2018-07-17 07:35:10.999,2018-07-16 06:59:59.999,2018-07-17 10:20:17.999,OBSTRUCCION,ELEMENTOS EXTRANOS O CONTAMINANTES EN EL FLUIDO,LIMPIAR,** JUL 16/2018 OT: 81011 PROGRAMO: WPARRAR EJ...
2,82516,R/P REPARAR VALVULA MANUAL DEL BY-PASS,2976936,CINME,2018-07-18 15:06:53.999,2018-11-26 06:59:59.999,2018-11-26 12:59:59.999,RETORNO NO ACCIONADO,FUGA,REPARAR,SOLICITANTE: REPARAR LA VALVULA MANUAL DE GUA...
3,82567,REPARAR AGUJAS DE REPUESTO,2976937,CINME,2018-07-18 15:50:47.999,2018-07-18 15:50:47.999,2020-12-04 10:44:12.000,"DANO EN RECUBRIMIENTO, ABOLLADURA, DEFORMACION","DESPLAZAMIENTO DEL TERRENO, DESLIZAMIENTO, ERO...",REPARAR,**JUL 18/2018 OT: 82567 PROGRAMO: WPARRAR EJEC...
4,83539,CORREGIR FUGA DE ACEITE CGI - U1,2976933,CINME,2018-07-19 14:43:19.999,2018-07-19 14:43:21.000,2018-08-08 10:42:17.000,FUGA,"FALLA EN ACCESORIO (VALVULA, SELLO, EMPAQUE, B...",CAMBIAR,SOLICITANTE: CORREGIR FUGA DE ACEITE EN EL CO...


In [71]:
# Guardo el csv
ots.to_csv('../Data/historico_OTs_recientes.csv')

# Tabla que relacione activo, fecha y reporte

In [50]:
viejos = hist_ots[['ASSETNUM','SLXNUMTOT','SLXFECINICIOOT','SLXFECFINOT','WORKLOG']]
nuevos = ots[['ASSETNUM','WONUM','ACTSTART','ACTFINISH','WORKLOG']]

In [51]:
viejos['ANTIGUEDAD'] = 'Antiguo'
nuevos['ANTIGUEDAD'] = 'Reciente'

<ipython-input-51-4f60f7d8b802>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viejos['ANTIGUEDAD'] = 'Antiguo'
<ipython-input-51-4f60f7d8b802>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['ANTIGUEDAD'] = 'Reciente'


In [52]:
viejos = viejos.rename(columns={'SLXNUMTOT': 'WONUM','SLXFECINICIOOT': 'ACTSTART','SLXFECFINOT':'ACTFINISH'})

In [53]:
viejos['ACTSTART'] = pd.to_datetime(viejos['ACTSTART'])
viejos['ACTFINISH'] = pd.to_datetime(viejos['ACTFINISH'])
viejos['ASSETNUM'] = viejos['ASSETNUM'].astype('object')

nuevos['ACTSTART'] = pd.to_datetime(nuevos['ACTSTART'])
nuevos['ACTFINISH'] = pd.to_datetime(nuevos['ACTFINISH'])
nuevos['ASSETNUM'] = nuevos['ASSETNUM'].astype('object')

<ipython-input-53-5446672fd909>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['ACTSTART'] = pd.to_datetime(nuevos['ACTSTART'])
<ipython-input-53-5446672fd909>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevos['ACTFINISH'] = pd.to_datetime(nuevos['ACTFINISH'])
<ipython-input-53-5446672fd909>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [54]:
todos = viejos.append(nuevos)
todos = todos.sort_values(by = ['ASSETNUM', 'ACTSTART'])

In [55]:
todos.head()

,ASSETNUM,WONUM,ACTSTART,ACTFINISH,WORKLOG,ANTIGUEDAD
9612,2976773.0,93702,2001-03-23 00:00:00.000,2001-03-23 00:00:00,NaN,Antiguo
2982,2976773.0,2192875,2012-11-16 00:00:00.000,2012-11-16 00:00:00,NaN,Antiguo
1053,2976774.0,1361312,2009-02-27 00:00:00.000,2009-02-27 00:00:00,NaN,Antiguo
8676,2976775,1739249,2020-03-10 16:08:49.999,2020-06-01 11:29:24,10/03/2020 Se verifica el funcionamiento del ...,Reciente
10687,2976775,1739249,2020-03-10 16:08:49.999,2020-06-01 11:29:24,REVISAR ARCHIVO ADJUNTO.,Reciente


In [49]:
todos.to_csv('relacion_ots_activo.csv')